In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

In [ ]:
import pandas as pd
import glob

# List to store standard deviations and filenames
std_devs = []

# Loop over each CSV file
for file in glob.glob("/content/B*.csv"):
    # Read the CSV file
    df = pd.read_csv(file)

    # Calculate the standard deviation of the SOH column
    soh_std = df['SOH'].std()

    # Append to the list as a tuple of (filename, standard deviation)
    std_devs.append((file, soh_std))

# Sort the list based on the standard deviation
sorted_files = sorted(std_devs, key=lambda x: x[1])

# Display sorted results
for file, std in sorted_files:
    print(f"{file}: {std}")


/content/B08.csv: 0.052853774130194354
/content/B06.csv: 0.07637558811806872
/content/B01.csv: 0.07821332845010749
/content/B03.csv: 0.09118976390970092
/content/B05.csv: 0.11001056381720095
/content/B07.csv: 0.11342750770012151
/content/B04.csv: 0.11575905639306033
/content/B02.csv: 0.13679096477364597


In [ ]:

files_to_concat = ['/content/B08.csv', '/content/B05.csv', '/content/B02.csv']

# Read and concatenate the files
concatenated_df = pd.concat([pd.read_csv(file) for file in files_to_concat], ignore_index=True)
# Save the concatenated DataFrame to a file in Colab's virtual environment
save_path = "/content/concatenated_df.csv"
concatenated_df.to_csv(save_path, index=False)

# Download the file to your local machine
from google.colab import files
files.download(save_path)

# Display the concatenated DataFrame
print(concatenated_df)



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

        terminal_voltage  terminal_current  temperature  charge_current  \
0               3.854275         -0.004909    23.620648          0.0000   
1               3.854246         -0.002896    23.620515          0.0002   
2               3.174256         -4.026186    23.693978          4.0000   
3               3.134106         -4.027110    23.947843          4.0000   
4               3.098808         -4.027657    24.323838          4.0000   
...                  ...               ...          ...             ...   
118644          3.700989         -0.004594    28.979655          0.0004   
118645          3.700967         -0.000881    28.905148          0.0004   
118646          3.701200         -0.002798    28.822990          0.0004   
118647          3.701104         -0.003812    28.757153          0.0004   
118648          3.701049         -0.003201    28.679696          0.0006   

        charge_voltage      time  capacity  cycle       SOH  
0                0.000     0.000  0.7

In [ ]:
df=concatenated_df
print(df.head())
print(df.shape)

   terminal_voltage  terminal_current  temperature  charge_current  \
0          3.854275         -0.004909    23.620648          0.0000   
1          3.854246         -0.002896    23.620515          0.0002   
2          3.174256         -4.026186    23.693978          4.0000   
3          3.134106         -4.027110    23.947843          4.0000   
4          3.098808         -4.027657    24.323838          4.0000   

   charge_voltage    time  capacity  cycle       SOH  
0           0.000   0.000   0.74593      1  0.372965  
1           3.868   9.328   0.74593      1  0.372965  
2           1.284  19.421   0.74593      1  0.372965  
3           1.247  28.750   0.74593      1  0.372965  
4           1.213  38.125   0.74593      1  0.372965  
(118649, 9)


In [ ]:


from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

input_vars = ['terminal_voltage', 'terminal_current', 'temperature', 'charge_current', 'charge_voltage', 'time', 'capacity', 'cycle']
output_var = 'SOH'

train, test = train_test_split(df, test_size=0.2, random_state=42)

train_input = train[input_vars]
train_output = train[output_var]
test_input = test[input_vars]
test_output = test[output_var]


# Define the Kalman filter model
class KalmanFilter(nn.Module):
    def __init__(self, dim):
        super(KalmanFilter, self).__init__()
        self.dim = dim
        self.F = nn.Parameter(torch.eye(dim))
        self.H = nn.Parameter(torch.eye(dim))
        self.Q = nn.Parameter(torch.eye(dim))
        self.R = nn.Parameter(torch.eye(dim))
        self.x_prior = nn.Parameter(torch.zeros(dim))
        self.P_prior = nn.Parameter(torch.eye(dim))

    def forward(self, z):
        # Prediction step
        x_prior = torch.matmul(self.F, self.x_prior.data)
        P_prior = torch.matmul(torch.matmul(self.F, self.P_prior), self.F.transpose(0, 1)) + self.Q

        # Update step
        HPHtR = torch.matmul(self.H, torch.matmul(P_prior, self.H.transpose(0, 1))) + self.R + 1e-8 * torch.eye(self.dim)
        K = torch.matmul(torch.matmul(P_prior, self.H.transpose(0, 1)), torch.inverse(HPHtR))
        x_posterior = x_prior + torch.matmul(K, z - torch.matmul(self.H, x_prior))
        P_posterior = torch.matmul(torch.eye(self.dim) - torch.matmul(K, self.H), P_prior)

        self.x_prior.data = x_posterior
        self.P_prior.data = P_posterior

        return x_posterior

# Initialize the model and perform training
model = KalmanFilter(dim=len(input_vars))
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    train_loss = 0.0
    for i in range(len(train_input)):
        input_tensor = torch.tensor(train_input.values[i], dtype=torch.float32)
        output_tensor = torch.tensor(train_output.values[i], dtype=torch.float32)

        output = model(input_tensor)
        loss = criterion(output, output_tensor)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        train_loss += loss.item()
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch+1, train_loss/len(train_input)))

# Perform testing and print the mean squared error
test_loss = 0.0
for i in range(len(test_input)):
    input_tensor = torch.tensor(test_input.values[i], dtype=torch.float32)
    output_tensor = torch.tensor(test_output.values[i], dtype=torch.float32)
    output = model(input_tensor)
    loss = criterion(output, output_tensor)
    test_loss += loss.item()
print('Test Loss: {:.6f}'.format(test_loss/len(test_input)))


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([8])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 1 	Training Loss: 401.391516
Epoch: 2 	Training Loss: 0.000962
Epoch: 3 	Training Loss: 0.000267
Epoch: 4 	Training Loss: 0.000111
Epoch: 5 	Training Loss: 0.000062
Epoch: 6 	Training Loss: 0.000041
Epoch: 7 	Training Loss: 0.000034
Epoch: 8 	Training Loss: 0.000028
Epoch: 9 	Training Loss: 0.000019
Epoch: 10 	Training Loss: 0.000017
Test Loss: 0.000000


In [ ]:
# Select an initial index from the test set to start predictions
initial_index = 10000  # change as desired
predictions = []
actual_values = []

# Use the initial input state from the test set
state = torch.tensor(test_input.values[initial_index], dtype=torch.float32)

# Generate 10 continuous predictions using the actual state each time
for step in range(10):
    # Perform prediction
    predicted_soh = model(state)

    # Assuming SOH is the last element in predicted_soh, extract it
    soh_value = predicted_soh[-1].item()  # adjust index if needed

    # Store the prediction and actual values
    predictions.append(soh_value)
    actual_values.append(test_output.values[initial_index + step] if initial_index + step < len(test_output) else None)

    # Update the state: use the actual next state from the test input
    if initial_index + step + 1 < len(test_input):
        state = torch.tensor(test_input.values[initial_index + step + 1], dtype=torch.float32)

# Print results
print("Step\tPredicted SOH\tActual SOH")
for step in range(10):
    actual = actual_values[step] if actual_values[step] is not None else "N/A"
    print(f"{step+1}\t{predictions[step]:.6f}\t\t{actual}")


Step	Predicted SOH	Actual SOH
1	0.831337		0.831160858
2	0.676454		0.676475209
3	0.829125		0.82899779
4	0.674398		0.674379944
5	0.817587		0.817406499
6	0.785718		0.785565168
7	0.790315		0.790126501
8	0.680961		0.680843254
9	0.806429		0.806268056
10	0.696164		0.696275605
